In [321]:
import finiteAutomata as lfa
import csv
import pandas as pd
from collections import OrderedDict as od
import string

In [322]:
# this uses the ascii table of uppercase letters to create the states
def check(state):
    if state == 82:
        state += 2
    else:
        state += 1
    return state

def read_csv(file_path):
    try:
        with open(file_path, newline='') as csvfile:
            reader = csv.reader(csvfile)
            lines = [line for line in reader if line]
            print(lines)
        return lines
    except FileNotFoundError:
        print(f'Arquivo {file_path} não encontrado.')
        exit(1)

def create_states(lines):
    states = {}
    initialState = 0
    state = 64 
    states['S'] = {}

    for line in lines:
        if line[0] != "<":
            for char in line:
                if not initialState:
                    state = check(state)
                    initialState = 1
                    states['S'].setdefault(char, []).append(chr(state))
                    states[chr(state)] = {}
                else:
                    state = check(state)
                    if state == 84:
                        states[chr(state - 2)].setdefault(char, []).append(chr(state))
                    else:
                        states[chr(state - 1)].setdefault(char, []).append(chr(state))
                    states[chr(state)] = {}
            initialState = 0
        else:
            rule = line.replace(' ::= ', ' | ').split(' | ')
            input_state = 'S' if rule[0][1] == 'S' else chr(state)
            createState = 0

            for m in range(1, len(rule)):
                if rule[m][0] != 'ε':
                    if rule[0][1] != rule[m][2]:
                        state = check(state)
                        states[chr(state)] = {}
                        createState = 1
                    states[input_state].setdefault(rule[m][0], []).append(chr(state))
                else:
                    states[input_state].setdefault(None, []).append('ε')

            if createState == 1:
                state = check(state)
                states[chr(state)] = {}

    return states

In [323]:
def extract_reserved_words(df):
    """
    Extract reserved words from a dataframe
    return: list of reserved words
    """
    reserved_words = [row for row in df.values for char in row if char.islower() and char != "<"]
    return reserved_words


In [324]:
def extract_terminals(df):
    """
    Extract terminals from a dataframe
    return: list of terminals
    """
    reserved_words = extract_reserved_words(df)
    terminal_letters = [c for row in reserved_words for char in row for c in char]
    return terminal_letters

In [325]:
def unique_terminal_letters(df):
    """
    Extract unique terminal letters from a dataframe
    return: list of unique terminal letters
    """
    terminal_letters = list(od.fromkeys((c for row in df.values for char in row for c in char if c.islower() and c != 'ε')))
    return terminal_letters

In [326]:

def populate_df(df):
    """
    Populate a dataframe with the terminal letters as columns
    return: dataframe
    """
    terminal_letters = unique_terminal_letters(df)
    df = pd.DataFrame(columns=['σ'] + [str(c) for c in terminal_letters])
    return df
    

In [327]:
def create_symbols(state):
    if state == 82:
        state += 2
    else:
        state += 1
    return state

In [328]:
def create_afnd(df):
    """
    Creates a NDFA from a dataframe
    
    
    returns: a data structure of a NDFA
    """
    if df["σ"].notnull().iloc[0]:
        # First cell of the "σ" column has a value
        print("First cell of the σ column has a value")
    else:
        # First cell of the "σ" column does not have a value
        print("First cell of the σ column does not have a value")
                             
                             

In [337]:
alphabet = list(string.ascii_uppercase)
print(alphabet[18])


S


In [330]:
df = pd.read_csv('./entrada.csv',  header=None)
df

afnd = populate_df(df)
afnd


,σ,s,e,n,t,a,o,i,u


In [331]:
lines = read_csv("./entrada.csv")
automaton_states = create_states(lines)

print("Autômato Finito Não-Determinístico:\n")
lfa.orderedStates(automaton_states)
lfa.determinize(automaton_states)
print("Autômato determinizado")
lfa.orderedStates(automaton_states)

[['se'], ['entao'], ['senao'], ['<S> ::= a<A> | e<A> | i<A> | o<A> | u<A>'], ['<A> ::= a<A> | e<A> | i<A> | o<A> | u<A> | Îµ']]
Autômato Finito Não-Determinístico:

A : {}
B : {}
C : {}
D : {}
E : {}
S : {'se': ['A'], 'entao': ['B'], 'senao': ['C'], '<S> ::= a<A> | e<A> | i<A> | o<A> | u<A>': ['D'], '<A> ::= a<A> | e<A> | i<A> | o<A> | u<A> | Îµ': ['E']}
Autômato determinizado
A : {}
B : {}
C : {}
D : {}
E : {}
S : {'se': ['A'], 'entao': ['B'], 'senao': ['C'], '<S> ::= a<A> | e<A> | i<A> | o<A> | u<A>': ['D'], '<A> ::= a<A> | e<A> | i<A> | o<A> | u<A> | Îµ': ['E']}
